In [1]:
import sys
import random
from pathlib import Path
from loguru import logger
from torch.utils.data import DataLoader
from torchvision import transforms as T
from hcmus.core import appconfig
from hcmus.data import CroppedCocoDataset
from hcmus.models.backbone import ResNetBackbone
from hcmus.models.backbone import DenseNetBackbone
from hcmus.models.backbone import CLIPBackbone
from hcmus.models.backbone import DinoBackbone
from hcmus.models.transfer import TransferNetwork
from hcmus.models.transfer import TransferTrainer
logger.remove()
logger.add(sys.stdout, level="INFO")

2025-08-21 07:04:38.077 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:
appconfig.MLFLOW_TRACKING_URI

'http://jimica.ddns.net:5050'

In [3]:
BATCH_SIZE = 16
N_EPOCH = 64
PATH_TO_DATASET = Path("/Volumes/Cucumber/Projects/datasets/raw/release-s2m-rp-2025-08-01")

In [4]:
transform_train = T.Compose([
    T.Lambda(lambda img: T.Resize(random.randint(32, 224))(img)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(
        brightness=0.3,    # ±20% brightness change
        contrast=0.1,      # ±20% contrast change
        saturation=0.1,    # ±20% saturation change
        hue=0.1           # ±10% hue shift
    ),
    T.RandomAffine(
        degrees=180,          # additional rotation control
        translate=(0.2, 0.2),  # 5% translation in both directions
        shear=10             # shear angle
    ),
    T.RandomResizedCrop(
        size=224,
        scale=(0.8, 1.2),
        ratio=(0.75, 1.3333)
    ),
    T.Resize((224, 224)),
    T.ToTensor()
])

transform_test = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

In [5]:
train_dataset = CroppedCocoDataset(PATH_TO_DATASET.joinpath("train"), transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = CroppedCocoDataset(PATH_TO_DATASET.joinpath("val"), transform=transform_test)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = CroppedCocoDataset(PATH_TO_DATASET.joinpath("test"), transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

2025-08-21 07:04:43.719 | INFO     | hcmus.data._torch_dataset_v2:__init__:197 - Skip category_id=101
2025-08-21 07:04:43.736 | INFO     | hcmus.data._torch_dataset_v2:__init__:197 - Skip category_id=101
2025-08-21 07:04:43.768 | INFO     | hcmus.data._torch_dataset_v2:__init__:197 - Skip category_id=101


In [ ]:
from mlflow.utils import name_utils
random_name = name_utils._generate_random_name()
output_dim = len(train_dataset.cat2idx)
backbones = [
    (CLIPBackbone, "ViT-B/32", 0.001),
    (CLIPBackbone, "ViT-B/16", 0.001),
    (DinoBackbone, "facebook/dinov2-small", 0.001),
    (DinoBackbone, "facebook/dinov2-base", 0.001),
    (DenseNetBackbone, "densenet121", 0.0001),
    # (DenseNetBackbone, "densenet169", 0.1),
    (ResNetBackbone, "resnet50", 0.0001),
    # (ResNetBackbone, "resnet101", 0.01),
]

for item in backbones:
    cls, name, lr = item
    backbone = cls(name)
    model = TransferNetwork(backbone, output_dim=train_dataset.num_classes)
    trainer = TransferTrainer(model, mlflow_experiment_name="/TransferLearningV2", mlflow_run_name=random_name + f"-{backbone._backbone_name}", learning_rate=lr)
    trainer.train(N_EPOCH, train_loader, val_loader, test_loader)


2025-08-21 07:04:45.186 | INFO     | hcmus.models.backbone:freeze:20 - Freeze backbone ViT-B/32.


Epoch 1/64: 100%|██████████| 166/166 [01:02<00:00,  2.68it/s, Loss=3.8719]


Start evaluating...


/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model saved to best_model.pt


...

Start evaluating...


/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 31/64:
  Train Loss: 0.6122
  Val Loss: 1.6700
  Val Accuracy: 0.6396
  Time: 174.64s
--------------------------------------------------


Epoch 32/64: 100%|██████████| 166/166 [01:02<00:00,  2.67it/s, Loss=0.2539]


Start evaluating...
